#**Feature** **Engineering**

**Loading Data**

In [ ]:
!pip install opendatasets --quiet 

import pandas as pd
import numpy as np
import opendatasets as od

import os

In [ ]:
od.download('https://www.kaggle.com/competitions/instacart-market-basket-analysis')

In [ ]:
#Checking the list of dataset in the directory
os.listdir('instacart-market-basket-analysis')

['orders.csv.zip',
 'departments.csv.zip',
 'products.csv.zip',
 'aisles.csv.zip',
 'order_products__train.csv.zip',
 'order_products__prior.csv.zip',
 'sample_submission.csv.zip']

In [ ]:
from zipfile import ZipFile

with ZipFile('instacart-market-basket-analysis/aisles.csv.zip') as f:
    f.extractall(path='market-basket')
with ZipFile('instacart-market-basket-analysis/orders.csv.zip') as f:
    f.extractall(path='market-basket')
with ZipFile('instacart-market-basket-analysis/departments.csv.zip') as f:
    f.extractall(path='market-basket')  
with ZipFile('instacart-market-basket-analysis/order_products__prior.csv.zip') as f:
    f.extractall(path='market-basket')
with ZipFile('instacart-market-basket-analysis/order_products__train.csv.zip') as f:
    f.extractall(path='market-basket')
with ZipFile('instacart-market-basket-analysis/products.csv.zip') as f:
    f.extractall(path='market-basket')
os.listdir('market-basket')

['order_products__prior.csv',
 'products.csv',
 'aisles.csv',
 '__MACOSX',
 'orders.csv',
 'departments.csv',
 'order_products__train.csv']

In [ ]:
aisle = pd.read_csv('./market-basket/aisles.csv')
dept = pd.read_csv('./market-basket/departments.csv')
products = pd.read_csv('./market-basket/products.csv')
train = pd.read_csv('./market-basket/order_products__train.csv', low_memory=False)
orders = pd.read_csv('./market-basket/orders.csv', low_memory=False)
prior = pd.read_csv('./market-basket/order_products__prior.csv', low_memory=False )

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

In [ ]:
reduce_mem_usage(orders)
reduce_mem_usage(prior)
reduce_mem_usage(train)

Memory usage of dataframe is 182.71 MB
Memory usage after optimization is: 45.68 MB
Decreased by 75.0%
Memory usage of dataframe is 989.82 MB
Memory usage after optimization is: 340.25 MB
Decreased by 65.6%
Memory usage of dataframe is 42.26 MB
Memory usage after optimization is: 13.20 MB
Decreased by 68.7%


**Data Preparation**

In [ ]:
#correcting spelling error
orders['eval_set'] = orders['eval_set'].replace(['pri'], 'prior')

In [ ]:
#merging prior dataset with orders
df = prior.merge(orders, on = 'order_id', how = 'inner').merge(products, on = 'product_id', how = 'left')

#dropping unrequired columns
df.drop(['aisle_id', 'department_id', 'eval_set'], axis=1, inplace=True)

#day_since_prior order contain nan value for first order. Imputing nan with 0
df = df.fillna(0)

df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name
0,2,33120,1,1,202279,3,5,9,8.0,Organic Egg Whites
1,2,28985,2,1,202279,3,5,9,8.0,Michigan Organic Kale
2,2,9327,3,0,202279,3,5,9,8.0,Garlic Powder
3,2,45918,4,1,202279,3,5,9,8.0,Coconut Butter
4,2,30035,5,0,202279,3,5,9,8.0,Natural Sweetener


In [ ]:
df.shape

(32434489, 10)

We have dataset with all user's previous orders details, which we will use to train our dataset. 

We will create features on the basis of users, products and products on user level.

**User features**

* total_orders_per_user = number of times a user bought from instacart
* total_products_per_user = number of products a user have bought 
* unique_products_per_user = number of unique products a user have bought 
* total_reorders_per_user = number of reordered products
* reorder_ratio_per_user = average reorder by a user
* avg_gap_in_orders_per_user = average number of days since prior order

In [ ]:
#total order per user
u1 = df.groupby('user_id')['order_number'].nunique().reset_index(name = 'total_orders_per_user')

#total products per user
u2 = df.groupby('user_id')['product_id'].count().reset_index(name = 'total_products_per_user')

#total unique products per user
u3 = df.groupby('user_id')['product_id'].nunique().reset_index(name = 'unique_products_per_user')

#total reorders
u4 = df.groupby('user_id')['reordered'].sum().reset_index(name = 'total_reorders_per_user')

#reorder ratio 
u5 = df.groupby('user_id')['reordered'].mean().reset_index(name = 'reorder_ratio_per_user')

#average gap between order
u6 = df.groupby('user_id')['days_since_prior_order'].mean().reset_index(name = 'avg_gap_in_orders_per_user')

In [ ]:
user = u1.merge(u2, how = 'left', on ='user_id').merge(u3, how = 'left', on = 'user_id').merge(u4, how = 'left', on= 'user_id').merge(u5, how = 'left', on = 'user_id').merge(u6,how = 'left', on ='user_id')

print(user.shape)
user.head()

(206209, 7)


,user_id,total_orders_per_user,total_products_per_user,unique_products_per_user,total_reorders_per_user,reorder_ratio_per_user,avg_gap_in_orders_per_user
0,1,10,59,18,41.0,0.694915,18.546875
1,2,14,195,102,93.0,0.476923,14.906250
2,3,12,88,33,55.0,0.625000,10.179688
3,4,5,18,17,1.0,0.055556,11.945312
4,5,4,37,23,14.0,0.378378,10.187500


**Product features**

* prod_reorder_ratio = Average reorder rate of a product
* num_orders_for_prod = number orders for a product
* avg_cart_order_of_product = average cart order for a product
* is_organic = if a product is organic
* is_vegan = if a product is vegan
* is_Gluten_free = if a product is gluten free

In [ ]:
#reorder_ratio
v1 = df.groupby('product_id')['reordered'].mean().reset_index(name = 'prod_reorder_ratio')

#number of products ordered
v2 = df.groupby('product_id')['order_id'].count().reset_index(name = 'num_orders_for_prod')

#avg cart to order 
v3 = df.groupby('product_id')['add_to_cart_order'].mean().round().reset_index(name = 'avg_cart_order_of_product')

v = df.groupby('product_id')['product_name'].min().reset_index()
v['is_organic'] = np.where(v['product_name'].str.contains('Organic'),1,0)
v['is_vegan'] = np.where(v['product_name'].str.contains('Vegan'),1,0)
v['is_Gluten-free'] = np.where(v['product_name'].str.contains('Gluten'),1,0)

In [ ]:
product = v1.merge(v2, how = 'left', on = 'product_id').merge(v3, how = 'left', on = 'product_id').merge(v, how = 'left', on = 'product_id')

print(product.shape)
product.head()

(49677, 8)


,product_id,prod_reorder_ratio,num_orders_for_prod,avg_cart_order_of_product,product_name,is_organic,is_vegan,is_Gluten-free
0,1,0.613391,1852,6.0,Chocolate Sandwich Cookies,0,0,0
1,2,0.133333,90,10.0,All-Seasons Salt,0,0,0
2,3,0.732852,277,6.0,Robust Golden Unsweetened Oolong Tea,0,0,0
3,4,0.446809,329,10.0,Smart Ones Classic Favorites Mini Rigatoni Wit...,0,0,0
4,5,0.600000,15,6.0,Green Chile Anytime Sauce,0,0,0


**User-Product Features**

* total_prod_by_user = number of orders for a product by a user
* prod_reorder_by_user = number of reorders for a product by a user
* prod_reorder_ratio_by_user = average reorder rate of a product by a user
* days_since_prior_prod_user = average days since prior order for a product of a user

In [ ]:
#number of times a product is ordered
up1 = df.groupby(['user_id', 'product_id'])['order_id'].count().reset_index(name = 'total_prod_by_user')

#number of times a product is reordered
up2 = df.groupby(['user_id', 'product_id'])['reordered'].sum().reset_index(name= 'prod_reorder_by_user')

#reordered ratio
up3 = df.groupby(['user_id', 'product_id'])['reordered'].mean().reset_index(name = 'prod_reorder_ratio_by_user')

#avearge_gap between product order
up4 = df.groupby(['user_id', 'product_id'])['days_since_prior_order'].mean().reset_index(name = 'days_since_prior_prod_user')

In [ ]:
user_prod = up1.merge(up2, how= 'left', on = ['user_id', 'product_id']).merge(up3, how= 'left', on = ['user_id', 'product_id']).merge(up4, how= 'left', on = ['user_id', 'product_id'])

print(user_prod.shape)
user_prod.head()

(13307953, 6)


,user_id,product_id,total_prod_by_user,prod_reorder_by_user,prod_reorder_ratio_by_user,days_since_prior_prod_user
0,1,196,10,9,0.900000,17.593750
1,1,10258,9,8,0.888889,19.562500
2,1,10326,1,0,0.000000,28.000000
3,1,12427,10,9,0.900000,17.593750
4,1,13032,3,2,0.666667,21.671875


In [ ]:
#merging all features

feature = user_prod.merge(product, how = 'left', on = 'product_id').merge(user, how = 'left', on = 'user_id')

print(feature.shape)
feature.head()

(13307953, 19)


,user_id,product_id,total_prod_by_user,prod_reorder_by_user,prod_reorder_ratio_by_user,days_since_prior_prod_user,prod_reorder_ratio,num_orders_for_prod,avg_cart_order_of_product,product_name,is_organic,is_vegan,is_Gluten-free,total_orders_per_user,total_products_per_user,unique_products_per_user,total_reorders_per_user,reorder_ratio_per_user,avg_gap_in_orders_per_user
0,1,196,10,9,0.900000,17.593750,0.776480,35791,4.0,Soda,0,0,0,10,59,18,41.0,0.694915,18.546875
1,1,10258,9,8,0.888889,19.562500,0.713772,1946,4.0,Pistachios,0,0,0,10,59,18,41.0,0.694915,18.546875
2,1,10326,1,0,0.000000,28.000000,0.652009,5526,4.0,Organic Fuji Apples,1,0,0,10,59,18,41.0,0.694915,18.546875
3,1,12427,10,9,0.900000,17.593750,0.740735,6476,5.0,Original Beef Jerky,0,0,0,10,59,18,41.0,0.694915,18.546875
4,1,13032,3,2,0.666667,21.671875,0.657158,3751,6.0,Cinnamon Toast Crunch,0,0,0,10,59,18,41.0,0.694915,18.546875


In [ ]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 19 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   user_id                     int64  
 1   product_id                  int64  
 2   total_prod_by_user          int64  
 3   prod_reorder_by_user        int8   
 4   prod_reorder_ratio_by_user  float64
 5   days_since_prior_prod_user  float16
 6   prod_reorder_ratio          float64
 7   num_orders_for_prod         int64  
 8   avg_cart_order              float64
 9   product_name                object 
 10  is_organic                  int64  
 11  is_vegan                    int64  
 12  is_Gluten-free              int64  
 13  total_orders                int64  
 14  total_products              int64  
 15  unique_products             int64  
 16  total_reorders              float64
 17  reorder_ratio               float64
 18  average_gap                 float16
dtypes: float16(2), floa

In [ ]:
reduce_mem_usage(feature)

Memory usage of dataframe is 1789.49 MB
Memory usage after optimization is: 610.58 MB
Decreased by 65.9%


In [ ]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 19 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   user_id                     int32   
 1   product_id                  int32   
 2   total_prod_by_user          int8    
 3   prod_reorder_by_user        int8    
 4   prod_reorder_ratio_by_user  float16 
 5   days_since_prior_prod_user  float16 
 6   prod_reorder_ratio          float16 
 7   num_orders_for_prod         int32   
 8   avg_cart_order              float16 
 9   product_name                category
 10  is_organic                  int8    
 11  is_vegan                    int8    
 12  is_Gluten-free              int8    
 13  total_orders                int8    
 14  total_products              int16   
 15  unique_products             int16   
 16  total_reorders              float16 
 17  reorder_ratio               float16 
 18  average_gap                 float16 
dty

In [ ]:
#merging train dataset with the orders data

train_orders = orders.merge(train, on = 'order_id', how = 'inner')
train_orders = train_orders.drop('eval_set', axis = 1)

In [ ]:
train_orders.shape

(1384617, 9)

In [ ]:
#creating a list of all users in the train dataset
user_list = train_orders.user_id.unique()

In [ ]:
#keeping only those users who are in the user list
data = feature[feature.user_id.isin(user_list)]

In [ ]:
final_data = data.merge(train_orders, on = ['user_id', 'product_id'], how = 'outer')

In [ ]:
#replacing nan values with the mean 
final_data.order_number.fillna(final_data.groupby('user_id')['order_number'].transform('mean'), inplace = True)
final_data.order_dow.fillna(final_data.groupby('user_id')['order_dow'].transform('mean'), inplace = True)
final_data.order_hour_of_day.fillna(final_data.groupby('user_id')['order_hour_of_day'].transform('mean'), inplace = True)
final_data.days_since_prior_order.fillna(final_data.groupby('user_id')['days_since_prior_order'].transform('mean'), inplace = True)

In [ ]:
final_data.reordered.value_counts()

1.0    828824
0.0    555793
Name: reordered, dtype: int64

In [ ]:
#since we only want to predict the reordered products, we will remove all first time ordered products
final_data = final_data[final_data['reordered'] != 0]

In [ ]:
#dropping irrelevant columns 
final_data = final_data.drop(['order_id', 'add_to_cart_order'], axis = 1)

In [ ]:
final_data.reordered.fillna(0, inplace = True)

In [ ]:
#dropping irrelevant columns 
final_data = final_data.drop('product_name', axis = 1)

In [ ]:
reduce_mem_usage(final_data)

Memory usage of dataframe is 1099.16 MB
Memory usage after optimization is: 484.92 MB
Decreased by 55.9%


In [ ]:
final_data

,user_id,product_id,total_prod_by_user,prod_reorder_by_user,prod_reorder_ratio_by_user,days_since_prior_prod_user,prod_reorder_ratio,num_orders_for_prod,avg_cart_order_of_product,is_organic,...,total_products_per_user,unique_products_per_user,total_reorders_per_user,reorder_ratio_per_user,avg_gap_in_orders_per_user,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.899902,17.593750,0.776367,35791.0,4.0,0.0,...,59.0,18.0,41.0,0.694824,18.546875,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888672,19.562500,0.713867,1946.0,4.0,0.0,...,59.0,18.0,41.0,0.694824,18.546875,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,28.000000,0.651855,5526.0,4.0,1.0,...,59.0,18.0,41.0,0.694824,18.546875,11.0,4.0,8.0,14.0,0.0
3,1,12427,10.0,9.0,0.899902,17.593750,0.740723,6476.0,5.0,0.0,...,59.0,18.0,41.0,0.694824,18.546875,11.0,4.0,8.0,14.0,0.0
4,1,13032,3.0,2.0,0.666504,21.671875,0.657227,3751.0,6.0,0.0,...,59.0,18.0,41.0,0.694824,18.546875,11.0,4.0,8.0,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,206209,43961,3.0,2.0,0.666504,23.328125,0.630371,55371.0,9.0,1.0,...,129.0,68.0,61.0,0.472900,18.234375,14.0,6.0,14.0,30.0,0.0
8474657,206209,44325,1.0,0.0,0.000000,9.000000,0.401123,3485.0,10.0,0.0,...,129.0,68.0,61.0,0.472900,18.234375,14.0,6.0,14.0,30.0,0.0
8474658,206209,48370,1.0,0.0,0.000000,30.000000,0.699219,3934.0,8.0,0.0,...,129.0,68.0,61.0,0.472900,18.234375,14.0,6.0,14.0,30.0,0.0
8474659,206209,48697,1.0,0.0,0.000000,9.000000,0.357666,9783.0,9.0,0.0,...,129.0,68.0,61.0,0.472900,18.234375,14.0,6.0,14.0,30.0,0.0


In [ ]:
#saving final data for model
final_data.to_pickle('final_data.pkl')